In [7]:
%pip install bayesian-optimization
%pip install hebo
%pip install GPyOpt
%pip install ax-platform scikit-learn numpy pandas matplotlib
%pip install pyGPGO
%pip install torch
%pip install antbo
%pip install ax-platform botorch emukit dragonfly-opt gpyopt smac

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.6 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.24.4 which is incompatible.
pymc3 3.11.6 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.12.0 which is incompatible.
crosspredict 1.1.9 requires pytest==6.2.4, but you have pytest 8.3.4 which is incompatible.
crosspredict 1.1.9 requires pytest-print==0.3.0, but you have pytest-print 1.

In [1]:
from hebo.design_space.design_space import DesignSpace
from hebo.optimizers.hebo import HEBO
from line_profiler import LineProfiler

profiler = LineProfiler()

def example_bo():
    # Define design space with controllable and contextual parameters
    config = [
        {'name': 'x', 'type': 'num', 'lb': 0, 'ub': 10},  # Controllable
        {'name': 'context', 'type': 'num', 'lb': 5, 'ub': 15},  # Uncontrollable (context)
    ]
    space = DesignSpace().parse(config)

    # Initialize HEBO with fixed context
    opt = HEBO(space, model_name='gp', rand_sample=4)

    # Optimization loop (context fixed during run)
    fixed_context = 10.0  # Set uncontrollable variable
    for _ in range(20):
        rec = opt.suggest(n_suggestions=1, fix_input={'context': fixed_context})
        y = -((rec['x'] - 5)**2) + 0.1 * fixed_context  # Example objective
        opt.observe(rec, y.array)

    print(opt.best_y)

profiler.add_function(example_bo)
profiler.enable()
example_bo()
profiler.disable()

profiler.print_stats()

jitter = 1e-07
jitter = 1e-06
jitter = 9.999999999999999e-06
jitter = 9.999999999999999e-05
jitter = 0.001
jitter = 0.01
jitter = 0.1
jitter = 1.0
jitter = 10.0
jitter is too large, give up fitting GP
jitter = 1e-07
jitter = 1e-06
jitter = 9.999999999999999e-06
jitter = 9.999999999999999e-05
jitter = 0.001
jitter = 0.01
jitter = 0.1
jitter = 1.0
jitter = 10.0
jitter is too large, give up fitting GP
jitter = 1e-07
jitter = 1e-06
jitter = 9.999999999999999e-06
jitter = 9.999999999999999e-05
jitter = 0.001
jitter = 0.01
jitter = 0.1
jitter = 1.0
jitter = 10.0
jitter is too large, give up fitting GP
jitter = 1e-07
jitter = 1e-06
jitter = 9.999999999999999e-06
jitter = 9.999999999999999e-05
jitter = 0.001
jitter = 0.01
jitter = 0.1
jitter = 1.0
jitter = 10.0
jitter is too large, give up fitting GP
jitter = 1e-07
jitter = 1e-06
jitter = 9.999999999999999e-06
jitter = 9.999999999999999e-05
jitter = 0.001
jitter = 0.01
jitter = 0.1
jitter = 1.0
jitter = 10.0
jitter is too large, give up fittin

In [2]:
profiler.print_stats()

Timer unit: 1e-09 s

Total time: 124.729 s
File: /tmp/ipykernel_11236/623481825.py
Function: example_bo at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def example_bo():
     8                                               # Define design space with controllable and contextual parameters
     9         1       1397.0   1397.0      0.0      config = [
    10         1       1956.0   1956.0      0.0          {'name': 'x', 'type': 'num', 'lb': 0, 'ub': 10},  # Controllable
    11         1       1397.0   1397.0      0.0          {'name': 'context', 'type': 'num', 'lb': 5, 'ub': 15},  # Uncontrollable (context)
    12                                               ]
    13         1      23397.0  23397.0      0.0      space = DesignSpace().parse(config)
    14                                           
    15                                               # Initialize HEBO with fixed context
    16         1   1334466

In [3]:
import GPyOpt
import numpy as np

# Define the objective: f(c, x) = - (x - c)^2 + 1.
def f(X):
    c = X[:, 0]
    x = X[:, 1]
    print("objective")
    return (- (x - c)**2 + 1.0).reshape(-1, 1)

# Define the search domain.
domain = [
    {'name': 'c', 'type': 'continuous', 'domain': (0, 1)},
    {'name': 'x', 'type': 'continuous', 'domain': (0, 1)}
]

# Lock the uncontrollable parameter "c" to 0.7.
fixed_features = {'x': 0.7}  # Index 0 corresponds to "c"

# Create the Bayesian optimization object.
opt = GPyOpt.methods.BayesianOptimization(
    f=f,
    domain=domain,
    acquisition_type='EI',
    maximize=True,
    num_cores=4
)

# Run sequential optimization (each iteration evaluates one new point).
opt.run_optimization(max_iter=20, context=fixed_features)
best_index = opt.Y.argmax()
print("GPyOpt best configuration:", opt.X[best_index])
print("GPyOpt best objective value:", opt.Y[best_index])


objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
objective
GPyOpt best configuration: [0.09646287 0.96135899]
GPyOpt best objective value: [-0.2519547]


In [4]:
import numpy as np

# Objective function: f(w, x, y, z) = sin(w) + cos(x) + y^2 - z
def objective_function(inputs):
    print(inputs)
    w = inputs[:, 0]  # Fixed context variable
    x = inputs[:, 1]  # Controlled variable
    y = inputs[:, 2]  # Controlled variable
    z = inputs[:, 3]  # Fixed context variable
    print("123 - ", (np.sin(w) + np.cos(x) + y**2 - z).reshape(-1, 1))
    return (np.sin(w) + np.cos(x) + y**2 - z).reshape(-1, 1)


In [5]:
import GPyOpt
import numpy as np

# Define the domain for all variables
domain = [
    {'name': 'w', 'type': 'continuous', 'domain': (-np.pi, np.pi), 'source': 'cfg'},
    {'name': 'x', 'type': 'continuous', 'domain': (-np.pi, np.pi), 'source': 'cfg'},
    {'name': 'y', 'type': 'continuous', 'domain': (-1.0, 1.0), 'source': 'cfg'},
    {'name': 'z', 'type': 'continuous', 'domain': (-1.0, 1.0), 'source': 'cfg'}
]

# Fixed context values for w and z
context = {'w': np.pi / 4, 'z': -0.5}

# Initialize the optimizer
optimizer = GPyOpt.methods.BayesianOptimization(
    f=objective_function,
    domain=domain,
    initial_design_numdata=1,
    model_type='GP',
    acquisition_type='EI',
    exact_feval=True,
    maximize=False,
)


optimizer.run_optimization(max_iter=1, context=context)

print(f"Fixed context variables: {context}")
print(f"Optimized variables: x = {optimizer.x_opt[0]}, y = {optimizer.x_opt[1]}")
print(f"Optimal objective value: {optimizer.fx_opt}")


[[ 0.41326949 -1.32868479 -0.83287793  0.53687515]]
123 -  [[0.79816932]]
[[ 0.78539816  0.74834324  0.0338794  -0.5       ]]
123 -  [[1.94107177]]
Fixed context variables: {'w': 0.7853981633974483, 'z': -0.5}
Optimized variables: x = 0.41326949416200254, y = -1.3286847887176483
Optimal objective value: 0.798169319776804


In [6]:
optimizer.get_evaluations()

(array([[ 0.41326949, -1.32868479, -0.83287793,  0.53687515],
        [ 0.78539816,  0.74834324,  0.0338794 , -0.5       ]]),
 array([[0.79816932],
        [1.94107177]]))